# API para consultar y registrar los juegos de Oferta en STEAM

# Crear base de datos
Vamos a crear una base de datos para nuestro proyecto. 

***Podemos omitir este paso si ya tenemos creada nuestra base de datos***

In [ ]:
import sqlite3
con = sqlite3.connect('games.db') # Warning: This file is created in the current directory
con.execute("CREATE TABLE games (title TEXT, price TEXT, tags TEXT, platforms TEXT, date TEXT)")
con.commit()

Vamos a comenzar por hacer el import de nuestro framework y definir nuestra primera ruta para listar los juegos almacenados en nuestra base de datos.

Será una petición GET, no lleva ningun parametro y nos dará como resultado una lista de los títulos de nuestros juegos (sin repetirse)

El formato de salida esta en JSON y es un elemento "games" y este tiene una lista de elementos que son el título del juego.

In [ ]:
from bottle import Bottle, run, request
import sqlite3

app = Bottle()

@app.route('/games', method='GET')
def get_list_games():
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()
    cursor.execute("SELECT DISTINCT title FROM games")
    result = cursor.fetchall()
    list_games=[]

    for title in result:
        list_games.append(title[0])
        
    return {"games": list_games}

run(app, host='0.0.0.0', port=3000, reloader=True, debug=True)

La siguiente ruta va a retornar una lista de juegos, pero ordenados por una fecha especificada por nosotros.

En este caso la ruta va a recibir un parametro que estará en la URL, podemos recibir parametros de diferentes formas:
A traves de la URL, en el cuerpo de la petición o en un header.

En este caso retornará un elemento **date** que será la fecha que pasemos por parametro y una lista de elementos que mostrará nombre y precio de los juegos guardados en esta fecha.

In [ ]:
from bottle import Bottle, run, request
import sqlite3

app = Bottle()

@app.route('/games', method='GET')
def get_list_games():
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()
    cursor.execute("SELECT DISTINCT title FROM games")
    result = cursor.fetchall()
    list_games=[]

    for title in result:
        list_games.append(title[0])
        
    return {"games": list_games}

@app.route('/games/date/<date>', method='GET')
def get_list_games(date):
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT title, price FROM games WHERE date='{date}'")
    result = cursor.fetchall()
    list_games=[]
    
    for title in result:
        gameobj = {title[0]:title[1]}
        list_games.append(gameobj)
        
    return {date : list_games}

run(app, host='0.0.0.0', port=3000, reloader=True, debug=True)

Nuestra siguiente ruta mostrará un historico de precios de un juego en especifico que pasaremos como parametro.

Como lo mencionamos previamente podemos pasar parametros de diferentes formas, aqui lo pasaremos en el cuerpo de la petición en formato JSON.

{"title": "mi juego"}

Este retornará un elemento con el nombre del juego y una lista de precios y fechas registradas.

In [ ]:
from bottle import Bottle, run, request
import sqlite3

app = Bottle()

@app.route('/games', method='GET')
def get_list_games():
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()
    cursor.execute("SELECT DISTINCT title FROM games")
    result = cursor.fetchall()
    list_games=[]

    for title in result:
        list_games.append(title[0])
        
    return {"games": list_games}

@app.route('/games/date/<date>', method='GET')
def get_list_games(date):
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT title, price FROM games WHERE date='{date}'")
    result = cursor.fetchall()
    list_games=[]
    
    for title in result:
        gameobj = {title[0]:title[1]}
        list_games.append(gameobj)
        
    return {date : list_games}

@app.route('/games/history', method='GET')
def get_game_history():
    data = request.json
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()

    title_game = data["title"]
    cursor.execute(f"SELECT price, date FROM games WHERE title='{title_game}'")
    result = cursor.fetchall()
    list_prices=[]
    for title in result:
        gameobj = {title[0]:title[1]}
        list_prices.append(gameobj)
    
    return {title_game: list_prices}

run(app, host='0.0.0.0', port=3000, reloader=True, debug=True)

Para nuestra siguiente ruta, definiremos una ruta donde actualizar datos(precio y fecha) de algun juego registrado.

En este caso también pasaremos parametro por el cuerpo de la petición.

Y retornará el titulo del juego y sus datos actualizados en formato JSON

In [ ]:
from bottle import Bottle, run, request
import sqlite3
import datetime

app = Bottle()

@app.route('/games', method='GET')
def get_list_games():
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()
    cursor.execute("SELECT DISTINCT title FROM games")
    result = cursor.fetchall()
    list_games=[]

    for title in result:
        list_games.append(title[0])
        
    return {"games": list_games}

@app.route('/games/date/<date>', method='GET')
def get_list_games(date):
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT title, price FROM games WHERE date='{date}'")
    result = cursor.fetchall()
    list_games=[]
    
    for title in result:
        gameobj = {title[0]:title[1]}
        list_games.append(gameobj)
        
    return {date : list_games}

@app.route('/games/history', method='GET')
def get_game_history():
    data = request.json
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()

    title_game = data["title"]
    cursor.execute(f"SELECT price, date FROM games WHERE title='{title_game}'")
    result = cursor.fetchall()
    list_prices=[]
    for title in result:
        gameobj = {title[0]:title[1]}
        list_prices.append(gameobj)
    
    return {title_game: list_prices}

@app.route('/games/update/price', method='POST')
def update_game_price():
    data = request.json
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()

    title_game = data["title"]
    title_price = data["price"]
    title_date = data["date"]
    now = datetime.date.today().strftime("%Y-%m-%d")

    cursor.execute(f"UPDATE games SET price='{title_price}', date='{now}' WHERE title='{title_game}' AND date='{title_date}'")
    connection.commit()
    
    return {title_game: {"price":title_price, "date":now }}

run(app, host='0.0.0.0', port=3000, reloader=True, debug=True)

Para la siguiente ruta eliminaremos un titulo por nombre y fecha, mezclando ambas formas de pasar parametros:

en la URL el titulo del juego, y en el cuerpo la fecha de este:

In [ ]:
from bottle import Bottle, run, request
import sqlite3
import datetime

app = Bottle()

@app.route('/games', method='GET')
def get_list_games():
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()
    cursor.execute("SELECT DISTINCT title FROM games")
    result = cursor.fetchall()
    list_games=[]

    for title in result:
        list_games.append(title[0])
        
    return {"games": list_games}

@app.route('/games/date/<date>', method='GET')
def get_list_games(date):
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT title, price FROM games WHERE date='{date}'")
    result = cursor.fetchall()
    list_games=[]
    
    for title in result:
        gameobj = {title[0]:title[1]}
        list_games.append(gameobj)
        
    return {date : list_games}

@app.route('/games/history', method='GET')
def get_game_history():
    data = request.json
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()

    title_game = data["title"]
    cursor.execute(f"SELECT price, date FROM games WHERE title='{title_game}'")
    result = cursor.fetchall()
    list_prices=[]
    for title in result:
        gameobj = {title[0]:title[1]}
        list_prices.append(gameobj)
    
    return {title_game: list_prices}

@app.route('/games/update/price', method='POST')
def update_game_price():
    data = request.json
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()

    title_game = data["title"]
    title_price = data["price"]
    title_date = data["date"]
    now = datetime.date.today().strftime("%Y-%m-%d")

    cursor.execute(f"UPDATE games SET price='{title_price}', date='{now}' WHERE title='{title_game}' AND date='{title_date}'")
    connection.commit()
    
    return {title_game: {"price":title_price, "date":now }}

@app.route('/games/delete/<title>', method='DELETE')
def delete_game(title):
    data = request.json
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()

    title_game = title
    #Si el cuerpo de la peticion esta vacio eliminará todos los elementos con el titulo del juego
    try:
        title_date = data["date"]
        cursor.execute(f"DELETE FROM games WHERE title='{title_game}' AND date='{title_date}'")
    except TypeError:
        cursor.execute(f"DELETE FROM games WHERE title='{title_game}'")
        
    connection.commit()
    
    return HTTPResponse(status=204)

run(app, host='0.0.0.0', port=3000, reloader=True, debug=True)


Por ultimo tenemos una ruta que ejecuta el proceso del web scraping de los juegos, y los inserta en la base de datos
y retornará una lista de los juegos que añadió.

In [ ]:
from bottle import Bottle, run, template, request
import sqlite3
import updategames
import datetime

app = Bottle()

@app.route('/games', method='GET')
def get_list_games():
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()
    cursor.execute("SELECT DISTINCT title FROM games")
    result = cursor.fetchall()
    list_games=[]

    for title in result:
        list_games.append(title[0])
        
    return {"games": list_games}

@app.route('/games/date/:date', method='GET')
def get_list_games(date):
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT title, price FROM games WHERE date='{date}'")
    result = cursor.fetchall()
    list_games=[]
    
    for title in result:
        gameobj = {title[0]:title[1]}
        list_games.append(gameobj)
        
    return {date : list_games}

@app.route('/games/history', method='GET')
def get_game_history():
    data = request.json
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()

    title_game = data["title"]
    cursor.execute(f"SELECT price, date FROM games WHERE title='{title_game}'")
    result = cursor.fetchall()
    list_prices=[]
    for title in result:
        gameobj = {title[0]:title[1]}
        list_prices.append(gameobj)
    
    return {title_game: list_prices}

@app.route('/games/update/price', method='POST')
def update_game_price():
    data = request.json
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()

    title_game = data["title"]
    title_price = data["price"]
    title_date = data["date"]
    now = datetime.date.today().strftime("%Y-%m-%d")

    cursor.execute(f"UPDATE games SET price='{title_price}', date='{now}' WHERE title='{title_game}' AND date='{title_date}'")
    connection.commit()
    
    return {title_game: {"price":title_price, "date":now }}

@app.route('/games/delete/:title', method='DELETE')
def delete_game(title):
    data = request.json
    connection = sqlite3.connect('games.db')
    cursor = connection.cursor()

    title_game = title
    #Si el cuerpo de la peticion esta vacio eliminará todos los elementos con el titulo del juego
    try:
        title_date = data["date"]
        cursor.execute(f"DELETE FROM games WHERE title='{title_game}' AND date='{title_date}'")
    except TypeError:
        cursor.execute(f"DELETE FROM games WHERE title='{title_game}'")
        
    connection.commit()
    
    return HTTPResponse(status=204)

@app.route('/games/update/list', method='GET')
def update_game_list():
    list_scrap = updategames.scrap_titles()
    updategames.update_game_list(list_scrap)

    connection = sqlite3.connect('games.db')
    
    now = datetime.date.today().strftime("%Y-%m-%d")

    cursor = connection.cursor()
    cursor.execute(f"SELECT * FROM games WHERE date='{now}'")
    result = cursor.fetchall()
    list_games=[]

    for title in result:
        list_games.append({title[0]: {"price": title[1], "tags":title[2], "platforms":title[3]}})
        
    return {now: list_games}

run(app, host='0.0.0.0', port=3000, reloader=True, debug=True)

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Licencia Creative Commons Atribución-CompartirIgual 4.0 Internacional</a>.